In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import yaml
import pickle
import numpy as np

In [2]:
# Load Data
data = pd.read_csv('/vepfs-sha/xiezixun/pregnant/data/input_set_123_test/clean_bug_input.csv', low_memory=False)
# 合并三列为一列
data['death'] = (data['foetal_death'].astype(int) | data['stillbirth'].astype(int) | data['death_7days'].astype(int)).astype(int)
# 删除原始列
data = data.drop(['foetal_death', 'stillbirth', 'death_7days', '母亲ID', '分娩时间'], axis=1)
# 将premature设置为0，1二分类任务
data['premature'] = data['premature'].replace([1, 2, 3], 1)

# 编码
# 时间编码：
encode_times = ['lmp', '首检日期', '孕产期']
for encode_time in encode_times:
    data[encode_time] = pd.to_datetime(data[encode_time])
    # 拆分为年、月、日三列
    data[encode_time+'_year'] = data[encode_time].dt.year
    data[encode_time+'_month'] = data[encode_time].dt.month
    data[encode_time+'_day'] = data[encode_time].dt.day
    data = data.drop(encode_time, axis=1)

# 'birth_weight', 'body_length', 'gw',这三个先不考虑，最后看哪种好一些
# 'foetal_death', 'stillbirth', 'death_7days' 三合一，death
target_cols = ['premature', 'low_BW', 'macrosomia', 'death', 'malformation','分娩方式','产后出血','SGA_S1', 'LGA_S1']
train_cols = [x for x in list(data.columns) if x not in target_cols]

# 其他分类列，顺序编码，0填充编码缺失值，将数值列用均值填充缺失值
categorical_columns = []
categorical_dims =  {}
for col in train_cols:
    if data[col].nunique() < 200:    # 分类量编码填充
        data[col], _ = pd.factorize(data[col])
        data[col] += 1  # pd.factorize默认编码nan = -1，然后0，1，2这样顺序编码，整体加1，方便后续embedding
        categorical_columns.append(col)
        categorical_dims[col] = data[col].nunique()
    else: # 连续量，用均值填充缺失值
        data[col] = data[col].fillna(data[col].mean())

# 获取分类变量的idx及种类数量，方便后续embedding
cat_idxs = [ i for i, f in enumerate(train_cols) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for f in train_cols if f in categorical_columns]

In [47]:
data['分娩方式'].value_counts()

分娩方式
0    2091760
1    1824593
Name: count, dtype: int64

In [19]:
convert_dict = {
    'premature':int,
    'low_BW':int,
    'macrosomia':int,
    'malformation':int,
    'SGA_S1':int,
    'LGA_S1':int
}

In [20]:
data = data.astype(convert_dict)

In [ ]:
target_cols = ['premature', 'low_BW', 'macrosomia', 'death', 'malformation','分娩方式','产后出血','SGA_S1', 'LGA_S1']

In [57]:
data = data[target_cols]

In [61]:
data.value_counts()

premature  low_BW  macrosomia  death  malformation  分娩方式  产后出血  SGA_S1  LGA_S1
0          0       0           0      0             1     0     0       0         1401869
                                                    0     0     0       0         1399485
                                                                        1          177330
1          1       0           0      0             0     0     0       0          105468
0          0       0           0      0             1     0     0       1          103508
                                                                                   ...   
1          0       0           1      0             1     1     0       2               2
           1       0           0      1             0     1     2       0               2
0          0       0           1      1             0     0     2       0               1
                                                                1       0               1
                   1 

In [66]:
print(data[(data['premature']==0) & (data['low_BW']==0) & (data['macrosomia']==1) & (data['death']==1) & (data['malformation']==1) & (data['分娩方式']==0) & (data['产后出血']==0) & (data['SGA_S1']==0) & (data['LGA_S1']==2)])

         premature  low_BW  macrosomia  death  malformation  分娩方式  产后出血  \
3299436          0       0           1      1             1     0     0   

         SGA_S1  LGA_S1  
3299436       0       2  


In [23]:
X = data[train_cols]
y = data[target_cols]

In [24]:
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.01, stratify=data[target_cols], random_state=42)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.